In [ ]:
%%capture 
!apt install gdal-bin python-gdal python3-gdal 
%pip install pandas fiona shapely pyproj
!apt install python3-rtree 
!pip install geopandas==0.10.0
!pip install descartes
!pip install scipy 
!pip install statsmodels
!pip install matplotlib
!pip install gdal
!pip install numpy
!pip install scikit-learn
!pip install statsmodels

In [ ]:
#Here we are importing the packages we need. 
import geopandas as gpd
import pandas as pd 
from geopandas.tools import sjoin
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import shape
from descartes import PolygonPatch
import time
import math
import scipy.stats as stats
import numpy as np
import os, sys
from pyproj import CRS, Transformer
import fiona

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import matplotlib as mpl
from math import floor

from shapely.ops import unary_union

import warnings
warnings.filterwarnings('ignore')

from osgeo import ogr, gdal,osr

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Navigate to folder where data is stored in the drive. 
%cd /content/drive/MyDrive/jameslab/

dirname = '/content/drive/MyDrive/jameslab/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/jameslab


In [ ]:
def read_data(fp):

    jack_pine_abundance = gpd.read_file(fp)
    return jack_pine_abundance


def convert_sfile(geometry,t):
    '''
    Takes a GeoSeries of 3D Multi/Polygons (has_z) and returns a list of 2D Multi/Polygons
    '''

    new_data= geometry[geometry['VALUE'] == t] #ROUND3 for age
    new_data['dissolvefield'] = 1
    new_data = new_data.dissolve(by='dissolvefield').dissolve()
    new_data['geometry'] = new_data.geometry.buffer(0.000001).unary_union.simplify(1)
    new_data['dissolvefield'] = 1
    new_data = new_data.dissolve(by='dissolvefield').dissolve()

    return new_data

In [ ]:

years = list(range(1985,2015+1)) #All years of fires considered

#read the file sent by Doriana and then convert it to epsg 4326
df_origin = read_data('doriana/fire_severity_selected_v003.shp').to_crs('EPSG:4326')
#sort by year to make easier 
df_origin = df_origin.sort_values('YEAR')

#loop through the years 
for y in years:

    try: 
      df1 = read_data('doriana/shp/'+str(y)+'_shp.shp') #open pre-made files in drive 
    except: 
      df1 = read_data('doriana/shp/'+str(y)+'_shp_n.shp') #case of repaired file 
    
    
    harvest = [] #List to collect area 
    names = [] 
    for geom,n in zip(list(df_origin['geometry']),list(df_origin['FIRE_ID'])): #Loop through each fire polygon 
        
        clip_harvest = df1.clip(geom,keep_geom_type=True) #Make sure clip does not produce lines 
        if len(list(clip_harvest['geometry'])) > 0: #If there is intersection b/t fire & disturbance area 
          
          geom2 = clip_harvest.to_crs('ESRI:102001') #convert to projected system 

          harvest.append(geom2.area[0]) #Calc area and append to list 
          names.append(n)
        else: 
          harvest.append(0) #If area disturbed is 0 in that year 
          names.append(n)

    df_origin[str(y)+'_AreaDisturbed'] = harvest #Create the new column and append data 

df_origin.to_file('doriana/area_disturbed2.geojson',driver="GeoJSON") #send geojson to drive 